In [1]:
import os
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from Bio import SeqIO
from Bio.Seq import Seq
from sqlalchemy import create_engine

if os.getcwd().endswith('notebook'):
    os.chdir('..')

In [2]:
sns.set(palette='colorblind', font_scale=1.3)

In [3]:
db_path = os.path.join(os.getcwd(), 'data/condensed_traits/db/seq.db')
engine = create_engine(f'sqlite+pysqlite:///{db_path}')

In [4]:
# Escherichia coli
species_taxid = 562

In [5]:
query = "select * from sequences where species_taxid = ? and sequence_type = 'tRNA'"
trnas = pd.read_sql(query, engine, params=(species_taxid,))

In [6]:
def add_metadata_column(attr):
    def fn(m):
        return json.loads(m)[attr]
    return fn

trnas['amino_acid'] = trnas['metadata_json'].apply(add_metadata_column('amino_acid'))
trnas['codon'] = trnas['metadata_json'].apply(add_metadata_column('codon'))

In [7]:
trnas.head()

,sequence_id,species_taxid,sequence_type,chromosome_id,location_json,strand,length,description,metadata_json,sequence,sequence_length,amino_acid,codon
0,lcl|U00096.3_trna_5,562,tRNA,U00096.3,"[[225381, 225457]]",+,77,lcl|U00096.3_trna_5 [gene=ileV] [locus_tag=b02...,"{""amino_acid"": ""Ile"", ""anticodon"": ""GAT"", ""cod...",AGGCTTGTAGCTCAGGTGGTTAGAGCGCACCCCTGATAAGGGTGAG...,77,Ile,ATC
1,lcl|U00096.3_trna_6,562,tRNA,U00096.3,"[[225500, 225575]]",+,76,lcl|U00096.3_trna_6 [gene=alaV] [locus_tag=b02...,"{""amino_acid"": ""Ala"", ""anticodon"": ""TGC"", ""cod...",GGGGCTATAGCTCAGCTGGGAGAGCGCCTGCTTTGCACGCAGGAGG...,76,Ala,GCA
2,lcl|U00096.3_trna_9,562,tRNA,U00096.3,"[[228928, 229004]]",+,77,lcl|U00096.3_trna_9 [gene=aspU] [locus_tag=b02...,"{""amino_acid"": ""Asp"", ""anticodon"": ""GTC"", ""cod...",GGAGCGGTAGTTCAGTCGGTTAGAATACCTGCCTGTCACGCAGGGG...,77,Asp,GAC
3,lcl|U00096.3_trna_10,562,tRNA,U00096.3,"[[236931, 237007]]",+,77,lcl|U00096.3_trna_10 [gene=aspV] [locus_tag=b0...,"{""amino_acid"": ""Asp"", ""anticodon"": ""GTC"", ""cod...",GGAGCGGTAGTTCAGTCGGTTAGAATACCTGCCTGTCACGCAGGGG...,77,Asp,GAC
4,lcl|U00096.3_trna_11,562,tRNA,U00096.3,"[[262871, 262946]]",+,76,lcl|U00096.3_trna_11 [gene=thrW] [locus_tag=b0...,"{""amino_acid"": ""Thr"", ""anticodon"": ""CGT"", ""cod...",GCCGATATAGCTCAGTTGGTAGAGCAGCGCATTCGTAATGCGAAGG...,76,Thr,ACG


In [8]:
trnas[['amino_acid', 'codon', 'sequence_length']].groupby(['amino_acid', 'codon']).count()

sequence_length
amino_acid codon                 
Ala        GCA                  3
           GCC                  2
Arg        AGA                  1
           AGG                  1
           CGG                  1
           CGT                  4
Asn        AAC                  4
Asp        GAC                  3
Cys        TGC                  1
Gln        CAA                  2
           CAG                  2
Glu        GAA                  4
Gly        GGA                  1
           GGC                  4
           GGG                  1
His        CAC                  1
Ile        ATC                  3
Leu        CTA                  1
           CTC                  1
           CTG                  4
           TTA                  1
           TTG                  1
Lys        AAA                  6
Met        ATG                  8
Phe        TTC                  2
Pro        CCA                  1
           CCC                  1
           CCG                  1
Sec(P)     TGA                  1
Ser        AGC                  1
           TCA                  1
           TCC                  2
           TCG                  1
Thr        ACA                  1
           ACC                  2
           ACG                  1
Trp        TGG                  1
Tyr        TAC                  3
Val        GTA                  5
           GTC                  2